In [ ]:
from datadoc_util import DataDocumentation as DD
import duckdb
import os

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict:  # try default location
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")

In [ ]:
ddoc = DD(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
df_grouping = ddoc.GROUPING_DF
df_segmentation = ddoc.SEGMENTATION_DF
df_colors = ddoc.COLORINGS_DF
df_win_inj_types = ddoc.WIN_INJ_TYPES_DF
df_events = ddoc.EVENTS_DF

In [ ]:
con = duckdb.connect('data_documentation.duckdb')

In [ ]:
# add experiment_type possible values as enum instead of string, sparing storage (but enum is static as of now, so adding 
# new enum types means re-building this database!)
enum_entries_exp_type = str(tuple(ddoc.GROUPING_DF.experiment_type.unique()))
con.execute("CREATE TYPE exp_type_enum AS ENUM " + enum_entries_exp_type)

In [ ]:
# add mouse_id as enum
enum_entries_mouse_id = str(tuple(sorted(list(set(ddoc.GROUPING_DF.mouse_id.unique()).union(set(ddoc.WIN_INJ_TYPES_DF.mouse_id.unique()))))))
con.execute("CREATE TYPE mouse_id_enum AS ENUM " + enum_entries_mouse_id)

In [ ]:
enum_entries_injection_type = str(tuple(ddoc.WIN_INJ_TYPES_DF.injection_type.unique()))
con.execute("CREATE TYPE injection_type_enum AS ENUM " + enum_entries_injection_type)

In [ ]:
enum_entries_window_type = str(tuple(ddoc.WIN_INJ_TYPES_DF.window_type.unique()))
con.execute("CREATE TYPE window_type_enum AS ENUM " + enum_entries_window_type)

In [ ]:
# merge window_side and injection_side
enum_entries_side = str(tuple(sorted(list(set(ddoc.WIN_INJ_TYPES_DF.window_side.unique()).union(set(ddoc.WIN_INJ_TYPES_DF.injection_side.unique()))))))
con.execute("CREATE TYPE side_enum AS ENUM " + enum_entries_side)

In [ ]:
"(\'" + ddoc.EVENTS_DF.event_type.unique()[0] + "\')"

In [ ]:
# event_type has only 1 unique value at the time of writing, need to remove comma from "('sz',)"
enum_entries_event_type = str(tuple(ddoc.EVENTS_DF.event_type.unique())) if len(ddoc.EVENTS_DF.event_type.unique()) > 1 else "(\'" + ddoc.EVENTS_DF.event_type.unique()[0] + "\')"
con.execute("CREATE TYPE event_type_enum AS ENUM " + enum_entries_event_type)

In [ ]:
enum_entries_interval_type = str(tuple(ddoc.EVENTS_DF.interval_type.unique()))
con.execute("CREATE TYPE interval_type_enum AS ENUM " + enum_entries_interval_type)

In [ ]:
enum_entries_segment_type = str(tuple(ddoc.SEGMENTATION_DF.interval_type.unique()))
con.execute("CREATE TYPE segment_type_enum AS ENUM " + enum_entries_segment_type)

In [ ]:
# add grouping table
con.execute("""
CREATE TABLE grouping (
folder VARCHAR, 
nd2 VARCHAR, 
labview VARCHAR, 
lfp VARCHAR, 
face_cam_last VARCHAR, 
nikon_meta VARCHAR, 
experiment_type exp_type_enum, 
day SIGNED, 
uuid UUID, 
day_index SIGNED, 
stim_length FLOAT, 
mouse_id mouse_id_enum
)
""")

In [ ]:
# populate grouping table
con.execute("INSERT INTO grouping SELECT * FROM df_grouping")

In [ ]:
# add segmentation table
con.execute("""
CREATE TABLE segmentation (
nd2 VARCHAR, 
interval_type segment_type_enum, 
frame_begin INT, 
frame_end INT
)
""")

In [ ]:
# populate segmentation table
con.execute("INSERT INTO segmentation SELECT * FROM df_segmentation")

In [ ]:
# add colors table
con.execute("""
CREATE TABLE colors (
mouse_id mouse_id_enum,  
color VARCHAR, 
red INT, 
green INT, 
blue INT
)
""")

In [ ]:
# populate colorings table
con.execute("INSERT INTO colors SELECT * FROM df_colors")

In [ ]:
# add win_inj_types table
con.execute("""
CREATE TABLE win_inj_types (
mouse_id mouse_id_enum, 
window_type window_type_enum, 
injection_type injection_type_enum, 
window_side side_enum, 
injection_side side_enum
)
""")

In [ ]:
# populate win_inj_types table
con.execute("INSERT INTO win_inj_types SELECT * FROM df_win_inj_types")

In [ ]:
# add events table
con.execute("""
CREATE TABLE events (
event_uuid UUID, 
window_type window_type_enum,
mouse_id mouse_id_enum, 
event_type event_type_enum, 
recording_uuid UUID, 
event_index INT, 
begin_frame INT, 
end_frame INT, 
interval_type interval_type_enum,
sz_index INT, 
has_lfp BOOLEAN
)
""")

In [ ]:
# populate events table
con.execute("INSERT INTO events SELECT * FROM df_events")

In [ ]:
con.close()